## Random Forest Hyperparameter Tuning (Recall)

In [24]:
# Load Libraries

import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split, StratifiedKFold
from sklearn.metrics import recall_score

from sklearn.ensemble import RandomForestClassifier

import optuna

In [26]:
# Load Data

df = pd.read_csv('bank_4.csv', index_col=0)

In [28]:
# Train / Test Split

X = df.drop(columns=['churn', 'complain', 'umap_1', 'umap_2'])
y = df['churn']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify=y, random_state=42)

In [32]:
# Hyperparameter tuning

def objective(trial):
    
    params = {
        'n_estimators': trial.suggest_int('n_estimators', 100, 2000),
        'max_features': trial.suggest_int('max_features', 4, 53),
        'max_depth': trial.suggest_int('max_depth', 2, 50),
        'min_samples_split': trial.suggest_int('min_samples_split', 2, 32),
        'min_samples_leaf': trial.suggest_int('min_samples_leaf', 1, 32)
    }

    model = RandomForestClassifier(
        **params,
        random_state=42,
        n_jobs=-1)
    
    threshold = 0.34
    
    skf = StratifiedKFold(n_splits=5, random_state=42, shuffle=True)
    scores = []
    
    for tr, te in skf.split(X_train, y_train):
        
        X_tr, X_te = X_train.iloc[tr], X_train.iloc[te]
        y_tr, y_te = y_train.iloc[tr], y_train.iloc[te]
        
        model.fit(X_tr, y_tr)
        prob = model.predict_proba(X_te)[:, 1]
        y_pred = np.where(prob < threshold, 0, 1)
        
        scores.append(recall_score(y_te, y_pred))
        
    return np.mean(scores)

In [34]:
study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=100, show_progress_bar=True)

[I 2024-07-07 18:51:57,646] A new study created in memory with name: no-name-32b7aa8e-a526-4004-b3da-f2386df73dad


  0%|          | 0/100 [00:00<?, ?it/s]

[I 2024-07-07 18:51:59,148] Trial 0 finished with value: 0.5914110429447853 and parameters: {'n_estimators': 105, 'max_features': 25, 'max_depth': 40, 'min_samples_split': 13, 'min_samples_leaf': 31}. Best is trial 0 with value: 0.5914110429447853.
[I 2024-07-07 18:52:05,020] Trial 1 finished with value: 0.5883435582822087 and parameters: {'n_estimators': 487, 'max_features': 51, 'max_depth': 7, 'min_samples_split': 31, 'min_samples_leaf': 1}. Best is trial 0 with value: 0.5914110429447853.
[I 2024-07-07 18:52:20,919] Trial 2 finished with value: 0.5981595092024541 and parameters: {'n_estimators': 1139, 'max_features': 30, 'max_depth': 12, 'min_samples_split': 28, 'min_samples_leaf': 25}. Best is trial 2 with value: 0.5981595092024541.
[I 2024-07-07 18:52:25,116] Trial 3 finished with value: 0.6018404907975461 and parameters: {'n_estimators': 285, 'max_features': 19, 'max_depth': 47, 'min_samples_split': 30, 'min_samples_leaf': 9}. Best is trial 3 with value: 0.6018404907975461.
[I 202

In [48]:
print("Best trial:", study.best_trial)
print("Best hyperparameters:", study.best_params)

Best trial: FrozenTrial(number=81, state=1, values=[0.6282208588957054], datetime_start=datetime.datetime(2024, 7, 7, 19, 14, 34, 399697), datetime_complete=datetime.datetime(2024, 7, 7, 19, 14, 39, 579163), params={'n_estimators': 294, 'max_features': 22, 'max_depth': 49, 'min_samples_split': 5, 'min_samples_leaf': 1}, user_attrs={}, system_attrs={}, intermediate_values={}, distributions={'n_estimators': IntDistribution(high=2000, log=False, low=100, step=1), 'max_features': IntDistribution(high=53, log=False, low=4, step=1), 'max_depth': IntDistribution(high=50, log=False, low=2, step=1), 'min_samples_split': IntDistribution(high=32, log=False, low=2, step=1), 'min_samples_leaf': IntDistribution(high=32, log=False, low=1, step=1)}, trial_id=81, value=None)
Best hyperparameters: {'n_estimators': 294, 'max_features': 22, 'max_depth': 49, 'min_samples_split': 5, 'min_samples_leaf': 1}
